In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
gscene.show_pose(crob.home_pose)

Please create a subscriber to the marker


## Door

In [4]:
from pkg.utils.code_scraps import *

In [5]:

##
# @class KnobTask
# @brief sweep action points in alphabetical order
# @remark   state_param: boolean vector of which each element represents if each waypoint is covered or not
#           node_item: number of covered waypoints
class KnobTask(WaypointTask):
    def __init__(self, oname, geometry, lever, action_points_dict, rebind_at_end=True, sub_binders_dict=None, tol=1e-3):
        self.lever = lever
        self.T0 = geometry.Toff
        self.link0 = geometry.link_name
        self.rebind_at_end = rebind_at_end
        WaypointTask.__init__(self, oname, geometry, action_points_dict,
                           sub_binders_dict=sub_binders_dict, tol=tol, one_direction=False)

    ##
    # @brief set object binding state and update location
    # @param binding BindingTransform
    # @param state_param list of done-mask
    def set_state(self, binding, state_param=None):
        self.binding = deepcopy(binding)
        self.state_param = np.zeros(len(self.action_points_order), dtype=np.bool)
        if self.binding.chain.handle_name is not None:
            if self.binding.chain.handle_name in self.action_points_order:
                self.state_param[:self.action_points_order.index(self.binding.chain.handle_name) + 1] = True
            else:
                raise (RuntimeError("Undefined handle {}".format(self.binding.chain.handle_name)))
        if binding.chain.handle_name is None:
            handle = self.action_points_dict[self.action_points_order[0]]
        else:
            handle = self.action_points_dict[binding.chain.handle_name]

        if self.rebind_at_end:
            if all(self.state_param):
                self.geometry.set_offset_tf(binding.T_lao[:3,3], binding.T_lao[:3,:3])
                self.geometry.set_link(binding.actor_link)
            else:
                self.geometry.set_offset_tf(self.T0[:3,3], self.T0[:3,:3])
                self.geometry.set_link(self.link0)

        self.update_sub_points()

    ##
    # @brief make constraints. by default, empty list.
    # @remark constraint is applied when using same binding
    # @param binding_from previous binding
    # @param binding_to next binding
    def make_constraints(self, binding_from, binding_to, tol=None):
        if binding_from is not None and binding_from.actor_name == binding_to.actor_name:
            return "Constraint not implemented yet. Use MoveitPlanner.incremental_constraint_motion=True"
        else:
            return []

In [ ]:

def add_door(pscene, hinge, dname = "door", center = (0.5,0,0.5), rpy = (0,0,0),
             dims = (0.05, 0.4, 0.5), hinge_point = (-0.025, 0.2, 0),
             door_ang = np.pi*1/4, door_div = 1, link_name="base_link", clearance=1e-3,
             frame_depth=0.0, frame_thickness=0.01, color = (0.8,0.8,0.8,1), add_frame=False):
    gscene = pscene.gscene
    Q0 = [0]*gscene.joint_num
    Tbh = hinge.get_tf(Q0)
    Tbhp = np.matmul(Tbh, SE3(np.identity(3), (0, 0, hinge.dims[2]/2+clearance)))
    Tbd = np.matmul(Tbhp, SE3(np.identity(3), (0, knob_offset, dims[2]/2)))
    door = gscene.create_safe(GEOTYPE.BOX, dname, link_name=link_name,
                              center=center, rpy=rpy, dims=dims,
                              fixed=False, collision=True, color=color)
    door_frame = gscene.create_safe(GEOTYPE.BOX, dname+"_frame", link_name=link_name,
                                    center=center, rpy=Rot2rpy(np.matmul(Rot_rpy(rpy), Rot_axis(2,np.pi/2))),
                                    dims=np.asarray(dims)[[2,1,0]],
                                    fixed=True, collision=False, color=(1,1,1,0.0), display=False)
    if add_frame:
        gscene.add_virtual_guardrail(door_frame, axis="xy", color=color, THICKNESS=frame_thickness,
                                     HEIGHT=dims[0]/2+frame_depth, margin=frame_thickness/2+clearance)

    door_r = pscene.create_binder(bname=door.name+"_hinge", gname=door.name, _type=HingeFramer,
                                  point=hinge_point)

    hinge_points = []
    for i_div in range(door_div+1):
        ang = door_ang/door_div*i_div
        hinge_points.append(HingeFrame("h{}".format(i_div), door_frame,
                                       np.asarray(hinge_point)[[2,1,0]],
                                       Rot2rpy(
                                           np.matmul(Rot_axis(2,np.pi/2).transpose(),
                                                     Rot_axis(3, -ang)))))

    door_s = pscene.create_subject(oname=door.name, gname=door_frame.name, _type=SweepTask,
                                  action_points_dict={hp.name: hp for hp in hinge_points},
                                  one_direction=False)
    return door, door_s

In [ ]:
def add_lever(pscene, knob,lname="lever", lever_ang=np.pi/4, knob_offset=(0.09), dims=(0.02, 0.2, 0.02),
              link_name="base_link", color=(0.8, 0.8, 0.8, 1), clearance = 1e-3):
    gscene = pscene.gscene
    Q0 = [0]*gscene.joint_num
    Tbk = knob.get_tf(Q0)
    Tbkp = np.matmul(Tbk, SE3(np.identity(3), (0, 0, knob.dims[2]/2+clearance)))
    Tbl = np.matmul(Tbkp, SE3(np.identity(3), (0, knob_offset, dims[2]/2)))
    lever = gscene.create_safe(GEOTYPE.BOX, lname, link_name=link_name,
                                    center=Tbl[:3,3], rpy=Rot2rpy(Tbl[:3,:3]), dims=dims,
                                    fixed=False, collision=True, color=color)
    lgrasp = Grasp2Point("gp", lever, (0, 0, 0), (np.pi/2, 0, -np.pi/2))
    lattach = PlaceFrame("cp", lever, (0, -knob_offset, -dims[2]/2), (0,0,0))

    lever_g = pscene.create_subject(oname=lname+"_grip", gname=lever.name, _type=CustomObject,
                                     action_points_dict={lgrasp.name: lgrasp, lattach.name: lattach})
    kpoint0 = pscene.create_binder(bname=knob.name + "_0", gname=knob.name, _type=AttachFrame,
                                   point=(0, 0, knob.dims[2]/2+clearance))
    kpoint1 = pscene.create_binder(bname=knob.name + "_1", gname=knob.name, _type=AttachFrame,
                                   point=(0, 0, knob.dims[2]/2+clearance), rpy=(0, 0, lever_ang))

    lever_plug = pscene.create_binder(bname=knob.name+"_plug", gname=lname, _type=KnobFramer,
                                   point=(0, -knob_offset, -dims[2]/2))
    rp1 = KnobFrame("r1", knob, (0, 0, knob.dims[2]/2+clearance), (0,0,0))
    rp2 = KnobFrame("r2", knob, (0, 0, knob.dims[2]/2+clearance), rpy=(0, 0, lever_ang))

    knob_s = pscene.create_subject(oname=knob.name, gname=knob.name, _type=KnobTask,
                                   action_points_dict={rp1.name: rp1, rp2.name: rp2},
                                   lever = lever_plug)
    return knob_s

In [5]:
door, door_s = add_door(pscene, center=(0.7,-0.2,0.5), color=(0.8,0.4,0.2,1))
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(-door.dims[0]/2-0.01,-door.dims[1]/2+0.05,0), rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=False, collision=True, color=(0.8,0.8,0.8,1), parent="door")

knob_s = add_lever(pscene, knob, dims=(0.02,0.1,0.02), knob_offset=0.04)

### prepare planner

In [6]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [7]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

### Plan

In [8]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

   Use a production WSGI server instead.
 * Debug mode: off
['door', 'knob', 'lever_grip']
(0, 0, 'knob')


In [9]:
goal_nodes = [(2, 2,'grip0')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True, add_homing=False)

Use 18/36 agents
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
result: 0 - (0, 0, 'knob')->(1, 0, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
result: 0 - (0, 0, 'knob')->(1, 0, 'knob') = fail
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
result: 0 - (0, 0, 'knob')->(1, 0, 'knob') = fail
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
try: 0 - (0, 0, 'knob')->(0, 0, 'grip0')
try transition motion
result: 0 - (0, 0, 'knob')->(1, 0, 'knob') = fail
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
result: 0 - (0, 0, 'knob')->(1, 0, 'knob') = fail
try: 0 - (0, 0, 'kno

branching: 12->16 (0.79/30.0 s, steps/err: 1(15.2280330658 ms)/0)
branching: 1->17 (0.79/30.0 s, steps/err: 65(178.395986557 ms)/0.00168692084604)
try: 10 - (0, 0, 'grip0')->(1, 0, 'grip0')
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
result: 0 - (0, 0, 'knob')->(0, 0, 'grip0') = success
transition motion tried: True
try: 13 - (0, 1, 'grip0')->(1, 1, 'grip0')
try: 19 - (0, 0, 'grip0')->(1, 0, 'grip0')
try: 15 - (0, 1, 'grip0')->(1, 1, 'grip0')
result: 2 - (0, 1, 'grip0')->(0, 1, 'knob') = success
branching: 0->12 (0.77/30.0 s, steps/err: 48(276.118040085 ms)/0.00166932804936)
result: 1 - (0, 0, 'grip0')->(0, 0, 'knob') = success
transition motion tried: True
branching: 2->18 (0.82/30.0 s, steps/err: 1(6.86693191528 ms)/0)
try: 6 - (0, 1, 'grip0')->(0, 2, 'grip0')
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
result: 10 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
try: 15 - (0, 1, 'grip0')->(1, 1, 'grip0')
try: 14 - (0, 1, 'grip0')->(0, 2, 'grip0')
br

try: 2 - (0, 1, 'grip0')->(1, 1, 'grip0')
try: 27 - (0, 1, 'knob')->(0, 2, 'knob')
branching: 0->31 (1.0/30.0 s, steps/err: 74(344.501018524 ms)/0.00165539249398)
try: 3 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 31 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
result: 16 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
result: 16 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
result: 27 - (0, 1, 'knob')->(0, 2, 'knob') = fail
collision
result: 15 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try: 20 - (0, 0, 'knob')->(1, 0, 'knob')
result: 2 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
constrained motion tried: False
try: 18 - (0, 1, 'knob')->(1, 1, 'knob')
try: 7 - (0, 0, 'grip0')->(1, 0, 'grip0')
try: 27 - (0, 1, 'knob')->(0, 2, 'knob')
try: 31 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 30 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
try: 6 - (0, 1, 'grip0')->(0, 1, 'knob')
try: 3 - (0, 0, 'grip0')->(1, 0, 'grip0')
Motion Plan Failure
branching: 30->32 (1.03/30.0 s, steps/err: 1(7.4200630188 ms)/0)
result

try: 16 - (0, 1, 'grip0')->(0, 1, 'knob')
result: 23 - (0, 0, 'knob')->(1, 0, 'knob') = fail
transition motion tried: True
try: 6 - (0, 1, 'grip0')->(0, 2, 'grip0')
try: 16 - (0, 1, 'grip0')->(0, 1, 'knob')
try transition motion
result: 22 - (0, 0, 'knob')->(0, 0, 'grip0') = success
Motion Plan Failure
try: 3 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 37 - (0, 1, 'knob')->(0, 2, 'knob') = fail
result: 24 - (0, 0, 'knob')->(0, 1, 'knob') = fail
result: 20 - (0, 0, 'knob')->(1, 0, 'knob') = fail
result: 16 - (0, 1, 'grip0')->(0, 1, 'knob') = success
try: 47 - (0, 0, 'grip0')->(0, 1, 'grip0')
try transition motion
try: 17 - (0, 0, 'knob')->(0, 1, 'knob')
branching: 16->46 (1.24/30.0 s, steps/err: 1(5.19585609436 ms)/0)
try: 16 - (0, 1, 'grip0')->(1, 1, 'grip0')
result: 10 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
result: 32 - (0, 1, 'grip0')->(0, 2, 'grip0') = fail
try: 23 - (0, 0, 'knob')->(0, 0, 'grip0')
try: 28 - (0, 1, 'grip0')->(1, 1, 'grip0')
try: 42 - (0, 1, 'grip0')->(0, 2, 'grip0

try: 61 - (0, 1, 'grip0')->(1, 1, 'grip0')
result: 31 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
try: 58 - (0, 1, 'grip0')->(0, 2, 'grip0')
try: 36 - (0, 1, 'knob')->(0, 2, 'knob')
try: 15 - (0, 1, 'grip0')->(0, 2, 'grip0')
try: 19 - (0, 0, 'grip0')->(0, 0, 'knob')
result: 58 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try transition motion
try: 23 - (0, 0, 'knob')->(1, 0, 'knob')
result: 61 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try: 49 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 36 - (0, 1, 'knob')->(0, 2, 'knob') = fail
try: 19 - (0, 0, 'grip0')->(0, 0, 'knob')
joint min
try: 35 - (0, 0, 'grip0')->(0, 0, 'knob')
constrained motion tried: False
result: 23 - (0, 0, 'knob')->(1, 0, 'knob') = fail
try transition motion
try: 9 - (0, 0, 'grip0')->(1, 0, 'grip0')
try: 3 - (0, 0, 'grip0')->(0, 0, 'knob')
result: 49 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
result: 19 - (0, 0, 'grip0')->(0, 0, 'knob') = success
branching: 19->62 (1.56/30.0 s, steps/err: 1(6.53791427612 ms)/0)
transition motio

...

result: 27 - (0, 1, 'knob')->(0, 1, 'grip0') = success
try: 93 - (0, 1, 'grip0')->(0, 2, 'grip0')
link transfer between same links: ['lever_grip'] - panda0
end
try transition motion
branching: 27->93 (2.19/30.0 s, steps/err: 7(132.189035416 ms)/0.00203489968712)
joint max
try: 92 - (0, 2, 'grip0')->(1, 2, 'grip0')
Motion Plan Failure
constrained motion tried: True
joint max
try: 27 - (0, 1, 'knob')->(0, 1, 'grip0')
constrained motion tried: False
result: 69 - (0, 1, 'grip0')->(0, 2, 'grip0') = fail
result: 59 - (0, 1, 'grip0')->(0, 2, 'grip0') = success
constrained motion tried: False
try: 82 - (1, 2, 'grip0')->(1, 1, 'grip0')
result: 92 - (0, 2, 'grip0')->(1, 2, 'grip0') = success
Motion Plan Failure
result: 86 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
link transfer between same links: ['knob'] - panda0
constrained motion tried: True
Motion Plan Failure
x
try: 95 - (1, 2, 'grip0')->(2, 2, 'grip0')
branching: 59->94 (2.23/30.0 s, steps/err: 80(709.476947784 ms)/1.51241849086e-05)
try: 8

try: 20 - (0, 0, 'knob')->(0, 1, 'knob')
try transition motion
branching: 35->106 (2.49/30.0 s, steps/err: 9(75.8311748505 ms)/0.00136480048283)
try: 42 - (0, 1, 'grip0')->(1, 1, 'grip0')
result: 57 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 30 - (0, 0, 'grip0')->(0, 1, 'grip0')
result: 30 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
result: 42 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try: 79 - (0, 2, 'grip0')->(0, 2, 'knob')
try: 107 - (0, 1, 'grip0')->(0, 2, 'grip0')
link transfer between same links: ['lever_grip'] - panda0
branching: 30->107 (2.52/30.0 s, steps/err: 1(7.48085975647 ms)/0)
result: 20 - (0, 0, 'knob')->(0, 1, 'knob') = fail
result: 106 - (0, 1, 'grip0')->(1, 1, 'grip0') = fail
try: 71 - (0, 1, 'knob')->(1, 1, 'knob')
Motion Plan Failure
try: 82 - (1, 2, 'grip0')->(1, 2, 'knob')
try: 66 - (0, 0, 'knob')->(1, 0, 'knob')
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
result: 71 - (0, 1, 'knob')->(1, 1, 'knob') = fail
result: 79 - (0, 2, 

result: 44 - (0, 0, 'knob')->(0, 0, 'grip0') = success
try: 63 - (0, 1, 'knob')->(0, 2, 'knob')
try transition motion
branching: 112->119 (2.85/30.0 s, steps/err: 44(97.8238582611 ms)/0.00191629033209)
branching: 44->120 (2.86/30.0 s, steps/err: 76(165.278911591 ms)/0.00159223433088)
try: 120 - (0, 0, 'grip0')->(0, 1, 'grip0')
try: 58 - (0, 1, 'grip0')->(0, 1, 'knob')
try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
result: 120 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
result: 63 - (0, 1, 'knob')->(0, 2, 'knob') = fail
branching: 120->121 (2.89/30.0 s, steps/err: 1(7.2660446167 ms)/0)
try: 76 - (0, 0, 'knob')->(0, 0, 'grip0')
result: 58 - (0, 1, 'grip0')->(0, 1, 'knob') = success
try transition motion
branching: 58->122 (2.89/30.0 s, steps/err: 1(8.5141658783 ms)/0)
joint max
try: 42 - (0, 1, 'grip0')->(1, 1, 'grip0')
constrained motion tried: False
Motion Plan Failure
transition motion tried: False
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 17 - (0, 0, 'knob')->(1, 0, 'knob')
re

======================= terminated 10: required answers acquired  (3.2/30.0) ===============================
result: 132 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
Motion Plan Failure
result: 110 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
result: 25 - (0, 0, 'knob')->(0, 0, 'grip0') = success
branching: 25->134 (3.22/30.0 s, steps/err: 10(58.1169128418 ms)/0.00116286815108)
======================= terminated 17: Stop called from other agent  (3.2/30.0) ===============================
======================= terminated 15: Stop called from other agent  (3.2/30.0) ===============================
======================= terminated 16: Stop called from other agent  (3.2/30.0) ===============================
transition motion tried: True
result: 126 - (0, 0, 'knob')->(0, 0, 'grip0') = success
branching: 126->135 (3.32/30.0 s, steps/err: 77(138.194084167 ms)/0.00134496373781)
======================= terminated 1: Stop called from other agent  (3.3/30.0) ===============================
end
cons

result: 43 - (0, 1, 'grip0')->(0, 2, 'grip0') = success
branching: 43->141 (3.6/30.0 s, steps/err: 80(474.309921265 ms)/1.39442439055e-05)
======================= terminated 4: Stop called from other agent  (3.6/30.0) ===============================
end
constrained motion tried: True
x
Goal reached
result: 131 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 131->142 (3.66/30.0 s, steps/err: 81(523.645877838 ms)/8.03309207585e-05)
======================= terminated 5: required answers acquired  (3.7/30.0) ===============================
========================== FINISHED (3.7 / 30.0 s) ==============================]


In [10]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [11]:
snode_schedule = tplan.get_best_schedule(at_home=False)
ppline.play_schedule(snode_schedule)

(0, 0, 'knob')->(0, 0, 'grip0')
(0, 0, 'grip0')->(0, 1, 'grip0')
(0, 1, 'grip0')->(0, 2, 'grip0')
(0, 2, 'grip0')->(1, 2, 'grip0')
(1, 2, 'grip0')->(2, 2, 'grip0')


In [23]:
snode = tplan.snode_dict[tplan.node_snode_dict[(0, 2, 'grip0')][0]]

In [24]:
pscene.set_object_state(snode.state)
gscene.show_pose(snode.state.Q)

In [25]:
gscene.show_motion(snode.traj)

In [28]:
door_hinge = pscene.actor_dict['door_hinge']

In [30]:
door_hinge.check_available(snode.state.Q)

True

In [ ]:

door_s.

In [31]:
tplan.node_dict

{(0, 0, 'grip0'): {(0, 0, 'knob'), (0, 1, 'grip0')},
 (0, 0, 'knob'): {(0, 0, 'grip0'), (0, 1, 'knob')},
 (0, 1, 'grip0'): {(0, 1, 'knob'), (0, 2, 'grip0')},
 (0, 1, 'knob'): {(0, 1, 'grip0'), (0, 2, 'knob')},
 (0, 2, 'grip0'): {(0, 1, 'grip0'), (0, 2, 'knob')},
 (0, 2, 'knob'): {(0, 1, 'knob'), (0, 2, 'grip0')},
 (1, 0, 'grip0'): {(1, 0, 'knob'), (1, 1, 'grip0')},
 (1, 0, 'knob'): {(1, 0, 'grip0'), (1, 1, 'knob')},
 (1, 1, 'grip0'): {(1, 1, 'knob'), (1, 2, 'grip0')},
 (1, 1, 'knob'): {(1, 1, 'grip0'), (1, 2, 'knob')},
 (1, 2, 'grip0'): set(),
 (1, 2, 'knob'): {(1, 1, 'knob'), (1, 2, 'grip0')},
 (2, 0, 'grip0'): {(2, 0, 'knob'), (2, 1, 'grip0')},
 (2, 0, 'knob'): {(2, 0, 'grip0'), (2, 1, 'knob')},
 (2, 1, 'grip0'): {(2, 1, 'knob'), (2, 2, 'grip0')},
 (2, 1, 'knob'): {(2, 1, 'grip0'), (2, 2, 'knob')},
 (2, 2, 'grip0'): {(2, 1, 'grip0'), (2, 2, 'knob')},
 (2, 2, 'knob'): {(2, 1, 'knob'), (2, 2, 'grip0')}}

In [32]:
tplan.node_dict_full

{(0, 0, 'grip0'): {(0, 0, 'knob'), (0, 1, 'grip0')},
 (0, 0, 'knob'): {(0, 0, 'grip0'), (0, 1, 'knob')},
 (0, 1, 'grip0'): {(0, 1, 'knob'), (0, 2, 'grip0')},
 (0, 1, 'knob'): {(0, 1, 'grip0'), (0, 2, 'knob')},
 (0, 2, 'grip0'): {(0, 1, 'grip0'), (0, 2, 'knob')},
 (0, 2, 'knob'): {(0, 1, 'knob'), (0, 2, 'grip0')},
 (1, 0, 'grip0'): {(1, 0, 'knob'), (1, 1, 'grip0')},
 (1, 0, 'knob'): {(1, 0, 'grip0'), (1, 1, 'knob')},
 (1, 1, 'grip0'): {(1, 1, 'knob'), (1, 2, 'grip0')},
 (1, 1, 'knob'): {(1, 1, 'grip0'), (1, 2, 'knob')},
 (1, 2, 'grip0'): {(1, 1, 'grip0'), (1, 2, 'knob')},
 (1, 2, 'knob'): {(1, 1, 'knob'), (1, 2, 'grip0')},
 (2, 0, 'grip0'): {(2, 0, 'knob'), (2, 1, 'grip0')},
 (2, 0, 'knob'): {(2, 0, 'grip0'), (2, 1, 'knob')},
 (2, 1, 'grip0'): {(2, 1, 'knob'), (2, 2, 'grip0')},
 (2, 1, 'knob'): {(2, 1, 'grip0'), (2, 2, 'knob')},
 (2, 2, 'grip0'): {(2, 1, 'grip0'), (2, 2, 'knob')},
 (2, 2, 'knob'): {(2, 1, 'knob'), (2, 2, 'grip0')}}

### push back

In [12]:
term_state = snode_schedule[-1].state
pscene.set_object_state(term_state)
gscene.show_pose(term_state.Q)
print(pscene.subject_name_list)
print(term_state.node)

['door', 'knob', 'lever_grip']
(2, 2, 'grip0')


In [13]:
goal_nodes = [(1, 1,'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['lever_grip'] - panda0
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
link transfer between same links: ['knob'] - panda0
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
Motion Plan Failure
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 0 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
result: 0 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2,

link transfer between same links: ['knob'] - panda0
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 4 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 1 - (1, 2, 'grip0')->(1, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
end
constrained motion tried: True
result: 1 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
try: 4 - (1, 2, 'grip0')->(1, 2, 'grip0')
branching: 0->5 (0.98/30.0 s, steps/err: 81(626.899957657 ms)/0.000155540352465)
link transfer between same links: ['knob'] - panda0
end
Motion Plan Failure
try: 5 - (1, 2, 'grip0')->(1, 2, 'knob')
constrained motion tried: True
link transfer between same links: ['lever_grip'] - panda0
end
constrained motion tried: True
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
result: 4 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
branching: 0->6 (1.02/30.0 s, steps/err: 81(514.789104462 ms)/0.000155540352465)
try: 6

try: 16 - (1, 2, 'grip0')->(1, 2, 'grip0')
branching: 0->17 (1.38/30.0 s, steps/err: 81(762.611150742 ms)/0.000155540352465)
try: 17 - (1, 2, 'grip0')->(1, 2, 'knob')
Motion Plan Failure
try: 0 - (2, 2, 'grip0')->(2, 2, 'knob')
link transfer between same links: ['knob'] - panda0
result: 15 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
result: 9 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
link transfer between same links: ['lever_grip'] - panda0
try: 11 - (1, 2, 'grip0')->(1, 2, 'knob')
Motion Plan Failure
Motion Plan Failure
try: 15 - (1, 2, 'grip0')->(2, 2, 'grip0')
result: 16 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
link transfer between same links: ['lever_grip'] - panda0
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
Not available:door-0
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
Not available:door-0
np_exclude:['h0']
Motion Plan Failure
try: 3 - (1, 2, 'grip0')->(1, 2, 'knob')
np_exclude:['h0']
result: 11 

try: 16 - (1, 2, 'grip0')->(1, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 16 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
end
constrained motion tried: True
x
result: 1 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 1->23 (1.89/30.0 s, steps/err: 81(512.478113174 ms)/0.000128042839292)
end
constrained motion tried: True
try: 2 - (1, 2, 'grip0')->(1, 2, 'grip0')
x
result: 9 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 9->24 (1.9/30.0 s, steps/err: 81(648.100137711 ms)/0.000128042839292)
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
end
constrained motion tried: True
result: 2 - (1, 2, 'grip0')->(1, 1, 'grip0') = fail
try: 24 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 0 - (2, 2, 'grip0')->(1, 

link transfer between same links: ['lever_grip'] - panda0
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
Motion Plan Failure
try: 24 - (2, 2, 'grip0')->(2, 2, 'grip0')
result: 30 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
try: 22 - (1, 2, 'grip0')->(1, 2, 'knob')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 0 - (2, 2, 'grip0')->(2, 2, 'knob') = fail
end
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
constrained motion tried: True
end
try: 0 - (2, 2, 'grip0')->(2, 2, 'grip0')
constrained motion tried: True
result: 22 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
result: 24 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
end
result: 0 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
try: 27 - (2, 2, 'grip0')->(2, 2, 'knob')
x
constrained motion tried: True
result: 9 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 0->33 (2.23/30.0 s, steps/err: 81(651.747941971 ms)/0.000155540352465)
Not available:door

...

try: 86 - (1, 2, 'grip0')->(2, 2, 'grip0')
============== Non-available transition: sample again =====================
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
result: 62 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
try: 124 - (2, 2, 'grip0')->(2, 2, 'grip0')
try: 34 - (2, 2, 'grip0')->(2, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
link transfer between same links: ['knob'] - panda0
Motion Plan Failure
result: 124 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
try: 135 - (1, 2, 'grip0')->(1, 2, 'knob')
result: 34 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
end
try: 16 - (1, 2, 'grip0')->(1, 2, 'knob')
constrained motion tried: True
link transfer between same links: ['lever_grip'] - panda0
end
x
constrained motion tried: True
link transfer between same links: ['lever_grip'] - panda0
end
result: 50 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
constrained motion tried: True
result: 89 - (2, 2, 'grip0')->(1, 2, 'grip0') = s

Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
============== Non-available transition: sample again =====================
============== Non-available transition: sample again =====================
Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
Not available:door-0
np_exclude:['h0']
bd_exclude:door_hinge
============== Non-available transition: sample again =====================
end
constrained motion tried: True
============== Non-available transition: sample again =====================
try: 6 - (1, 2, 'grip0')->(2, 2, 'grip0')
try: 115 - (1, 2, 'grip0')->(1, 2, 'knob')
result: 155 - (2, 2, 'grip0')->(1, 2, 'grip0') = success
branching: 155->170 (7.94/30.0 s, steps/err: 81(588.092088699 ms)/0.000139210451573)
link transfer between same links: ['lever_grip'] - panda0
Motion Plan Failure
try: 170 - (1, 2, 'grip0')->(1, 2, 'grip0')
link transfer between same links: ['knob'] - panda0
result: 115 - (1, 2, 'grip0')->(1, 2, 'knob') = fail
Motion Plan Failure
end


end
try: 163 - (1, 2, 'grip0')->(2, 2, 'grip0')
x
constrained motion tried: True
result: 22 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
x
branching: 22->177 (8.22/30.0 s, steps/err: 81(686.458826065 ms)/0.000128042839292)
result: 142 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
Not available:door-0
branching: 142->178 (8.22/30.0 s, steps/err: 81(704.078912735 ms)/0.00018326876969)
try: 104 - (2, 2, 'grip0')->(2, 2, 'grip0')
np_exclude:['h0']
bd_exclude:door_hinge
link transfer between same links: ['knob'] - panda0
============== Non-available transition: sample again =====================
Motion Plan Failure
Not available:door-0
result: 104 - (2, 2, 'grip0')->(2, 1, 'grip0') = fail
try: 119 - (1, 2, 'grip0')->(1, 2, 'knob')
np_exclude:['h0']
link transfer between same links: ['lever_grip'] - panda0
bd_exclude:door_hinge
Motion Plan Failure
Not available:door-0
np_exclude:['h0']
============== Non-available transition: sample again =====================
bd_exclude:door_hinge
result: 11

end
constrained motion tried: True
x
result: 93 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 93->190 (8.69/30.0 s, steps/err: 81(644.090890884 ms)/0.00018326876969)
end
constrained motion tried: True
x
result: 62 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 62->191 (8.71/30.0 s, steps/err: 81(569.213151932 ms)/0.00018326876969)
end
constrained motion tried: True
x
result: 117 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 117->192 (8.76/30.0 s, steps/err: 81(611.049890518 ms)/0.00018326876969)
end
constrained motion tried: True
x
end
result: 100 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
constrained motion tried: True
branching: 100->193 (8.82/30.0 s, steps/err: 81(508.264064789 ms)/0.00018326876969)
x
result: 90 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 90->194 (8.83/30.0 s, steps/err: 81(490.211009979 ms)/0.00018326876969)
end
constrained motion tried: True
x
result: 121 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 121->195 (8

======================= terminated 3: node queue empty 16th  (9.6/30.0) ===============================
======================= terminated 4: node queue empty 17th  (9.6/30.0) ===============================
======================= terminated 1: node queue empty 18th  (9.6/30.0) ===============================
========================== FINISHED (9.7 / 30.0 s) ==============================]


In [14]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

IndexError: list index out of range

generate table - Geometry
generate table - Handle
 generate table - Object
generate table - Binder
generate table - PlanConditions
generate table - PlanList
button clicked
(2, 2, 'grip0')->(1, 2, 'grip0')
button action done
